<a href="https://colab.research.google.com/github/Gangadharan-S/Deployment-of-News-Article-Categorization-Application-Using-AWS-Services__-FinalProject-./blob/main/News_Article_Categorization_(Final_Project).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deployment of News Article Categorization Application Using AWS Services :


In [ ]:
# Imports libraries:-
import os
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
from torch.cuda.amp import autocast, GradScaler

In [ ]:
# Define CSV file path & Directory Setup:-
save_path = "/content/drive/MyDrive/train.csv"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
# Now you can safely write to train.csv:-
with open(save_path, "w") as f:
    f.write("sample,data,to,save\n1,2,3,4")

In [ ]:
# Load & Prepare Dataset and Extract texts & labels:-
dataset = load_dataset("ag_news")
texts = [item["text"] for item in dataset["train"]]
labels = [item["label"] for item in dataset["train"]]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
# Sample & Split Data:-
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
for train_idx, test_idx in splitter.split(texts, labels):
    small_train_texts = [texts[i] for i in train_idx[:10000]]
    small_train_labels = [labels[i] for i in train_idx[:10000]]
    small_test_texts = [texts[i] for i in test_idx[:5000]]
    small_test_labels = [labels[i] for i in test_idx[:5000]]

In [ ]:
# Load Model & Tokenizer:-
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name = "prajjwal1/bert-tiny"
# Small & fast alternative:-
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Encode Labels:-
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(small_train_labels)
y_test = label_encoder.transform(small_test_labels)

In [ ]:
# Tokenize Inputs:-
def tokenize(texts, tokenizer, max_length=512):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
train_encodings = tokenize(small_train_texts, tokenizer)
test_encodings = tokenize(small_test_texts, tokenizer)

In [ ]:
# Define a custom PyTorch Dataset class for news data:-
class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}, self.labels[idx]
# Create dataset objects for training & testing:-
train_dataset = NewsDataset(train_encodings, y_train)
test_dataset = NewsDataset(test_encodings, y_test)

In [ ]:
# Create DataLoaders:-
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Set Device Training Setup:-
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
# Loss & Optimizer:-
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# Training Loop:-
def train_model(model, train_loader, criterion, optimizer, epochs=3):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch, labels in train_loader:
            batch = {key: val.to(device) for key, val in batch.items()}
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(**batch).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")
        torch.save(model.state_dict(), f"/content/drive/MyDrive/news/bert_tiny_news_classifier_epoch{epoch+1}.pth")
        print(f"Model saved for epoch {epoch+1}")

# Train the Model:-
train_model(model, train_loader, criterion, optimizer)

Epoch 1, Loss: 0.6827950302404336
Model saved for epoch 1
Epoch 2, Loss: 0.48191800323157263
Model saved for epoch 2
Epoch 3, Loss: 0.38492596263702683
Model saved for epoch 3


In [ ]:
# Save Final Model
torch.save(model.state_dict(), "/content/drive/MyDrive/news/bert_tiny_news_classifier_final.pth")
print("TinyBERT model trained and saved successfully!")

TinyBERT model trained and saved successfully!


In [ ]:
from sklearn.metrics import classification_report

# Evaluate Model:-
model.eval()
predictions, true_labels = [], []

for batch, labels in test_loader:
    batch = {key: val.to(device) for key, val in batch.items()}
    labels = labels.to(device)
    with torch.no_grad():
        outputs = model(**batch).logits
    preds = torch.argmax(outputs, dim=1)
    predictions.extend(preds.cpu().numpy())
    true_labels.extend(labels.cpu().numpy())

# Print Classification Report:-
print(classification_report(true_labels, predictions))

              precision    recall  f1-score   support

           0       0.92      0.86      0.89      1267
           1       0.95      0.97      0.96      1240
           2       0.84      0.82      0.83      1252
           3       0.83      0.89      0.86      1241

    accuracy                           0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000



In [ ]:
# Define Label Mapping:-
label_mapping = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Science/Tech"
}

# Prediction Function:-
def predict_label(model, tokenizer, text):
    model.eval()  # Ensure evaluation mode
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    with torch.no_grad():
        output = model(**inputs).logits
        pred_label = torch.argmax(output, dim=1).cpu().item()

    return label_mapping[pred_label]  # Map numeric label to category

# Example Prediction:-
sample_texts = [
    "Stock markets are experiencing a huge crash this week.",
    "The Mars rover has discovered signs of ancient life.",
    "Argentina wins FIFA World Cup 2026 after an intense match!",
    "New AI technology is revolutionizing the tech industry."
]

for text in sample_texts:
    predicted_label = predict_label(model, tokenizer, text)
    print(f'📌 Input: {text}\n🔹 Predicted Class: {predicted_label}\n')

📌 Input: Stock markets are experiencing a huge crash this week.
🔹 Predicted Class: Business

📌 Input: The Mars rover has discovered signs of ancient life.
🔹 Predicted Class: Science/Tech

📌 Input: Argentina wins FIFA World Cup 2026 after an intense match!
🔹 Predicted Class: Sports

📌 Input: New AI technology is revolutionizing the tech industry.
🔹 Predicted Class: Science/Tech

